In [1]:
import sys
from IPython.core.debugger import set_trace
print(sys.version)

3.7.3 (default, Mar 27 2019, 16:54:48) 
[Clang 4.0.1 (tags/RELEASE_401/final)]


In [2]:
from jinja2 import Template
from datetime import date
import json
import re

In [3]:
appID = 'LongGames.Alarm.ios.swift'#'com.byteorbit.malcolm.Calculator'
sessionID = '1591805093'#'1591639968'#'1591519569'

# 1. pull up and save the session json here
with open('{}/{}.json'.format(appID, sessionID)) as f:
    data = json.load(f)

if not data:
    assert False
    
logs = data['log_data']

In [4]:
# 1. check what kind of UIElement is this acti
# to check for button
# "viewDetails": "<Calculator.CalculatorBtn: 0x7ff311b07f40; baseClass = UIButton;
#  frame = (95.6667 264.333; 92 86); opaque = NO; tag = 2; layer = <CALayer: 0x60000032ae20>>"
# TODO:-// more?
def findXCUIElement(log):
    pattern = "(?<=baseClass = ).*?(?=;)"
    classs = re.search(pattern, log['viewDetails'])
    #print(log['viewDetails'])
    #print(log['superView'])
    #print(log['receiverSubview'])

    if classs:
        if classs.group(0) == 'UIButton':
            el_type = 'XCUIElementTypeButton'
        elif classs.group(0) == 'UILabel':
            el_type = 'XCUIElementTypeStaticText'
    elif "<UILabel:" in log['viewDetails']:
        el_type = "XCUIElementTypeStaticText"
    elif "<UIButton" in log['viewDetails']:
        el_type = "XCUIElementTypeButton"
    elif "TableViewCell" in log['superView'] or "<UITableViewLabel:" in log['viewDetails']:
        el_type = "XCUIElement"
    elif "<_UIAlertControllerActionView" in log['viewDetails']:
        el_type = 'XCUIElementTypeButton'
    elif "<_UIBackButtonContainerView" in log['receiverSubview']:
        el_type = 'XCUIElementTypeBackButton'
    elif "<_UIButtonBarButton:" in log['viewDetails']:
        el_type = 'XCUIElementTypeButton'
        
    #print(log['viewDetails'])
    #print(el_type)
    
    assert (el_type is not None)
    return el_type

def getVal(i, log):
    #set_trace()
    pattern = "(?<=text = ').*?(?=';)"
    
    #print(log['viewDetails'])
    
    vd = log['viewDetails'].replace("\n", " ")
    rv = log['receiverSubview'].replace("\n", " ")

    if re.search(pattern, rv):
        val = re.search(pattern, rv).group(0)
    elif re.search(pattern, vd):
        val = re.search(pattern, vd).group(0)
    elif ("<_UIAlertControllerActionView:" in vd):
        val = re.search("""(?<=Title = \\").*?(?=\\")""", vd).group(0)
    elif log['accessibilityLabell']:
        val = log['accessibilityLabell']
    assert (val is not None)
    return val

def evaluateElement(i, log, s):
    sentences = s
    #print(log["isAnAssertionLog"])
    if log["isAnAssertionLog"] == "false":
        #print(i)
        t= Template("XCTAssertTrue(el{{idx}}.exists)").render(idx=i)
        sentences.append(t)
        #print(t)
        t = Template("el{{idx}}.tap()").render(idx=i)
        sentences.append(t)
        #print(t)
    else:
        # this is an assertion log
        for key in log["assert_items"]:
            #print(key)
            if re.search("(?<=).*?(?=:)", key) and re.search("(?<=).*?(?=:)", key).group(0) == "Enabled":
                #set_trace()
                #print("en")
                regex = """(?<=: ).*"""
                test_str = log["assert_items"][key]
                #print(test_str)
                if re.search(regex, test_str):
                    #print("en2")
                    val_to_match = re.search(regex, log["assert_items"][key]).group(0)
                    t = Template("""XCTAssert(el{{idx}}.isEnabled == {{val}})""").render(idx=i, val=val_to_match)
                    sentences.append(t)
            if re.search("(?<=).*?(?=:)", key) and re.search("(?<=).*?(?=:)", key).group(0) == "Selected":
                regex = """(?<=: ).*"""
                test_str = log["assert_items"][key]
                if re.search(regex, test_str):
                    val_to_match = re.search(regex, log["assert_items"][key]).group(0)
                    t = Template("""XCTAssert(el{{idx}}.isSelected == {{val}})""").render(idx=i, val=val_to_match)
                    sentences.append(t)
            
            if re.search("(?<=).*?(?=:)", key) and re.search("(?<=).*?(?=:)", key).group(0) == "State of the Element":
                #TODO:-//
                """
                static var normal: UIControl.State
                    The normal, or default state of a control—that is, enabled but neither selected nor highlighted.

                static var highlighted: UIControl.State
                    Highlighted state of a control.

                static var disabled: UIControl.State
                    Disabled state of a control.

                static var selected: UIControl.State
                    Selected state of a control.

                static var focused: UIControl.State
                    Focused state of a control.

                static var application: UIControl.State
                    Additional control-state flags available for application use.

                static var reserved: UIControl.State
                    Control-state flags reserved for internal framework use.
                """
            
            #if re.search("(.*: \()", key) and re.search("(.*: \()", key).group(0) == "Receiver Subview: (":
            #    regex = "(?<=text = ).*?(?=;)"
            #    test_str = log["assert_items"][key]
            #     if re.search(regex, test_str):
            #        val_to_match = re.search(regex, log["assert_items"][key]).group(0)
            #        val_to_match = val_to_match[1:-1]
            #        
            #        t = Template("""let el{{idx}}s = app.staticTexts.containing(NSPredicate(format: "label LIKE[c] '{{val_to_match}}'"))""").render(idx=i, val_to_match=val_to_match)
            #        sentences.append(t)
                    
            if re.search("(.*: \()", key) and re.search("(.*: \()", key).group(0) == "Receiver Subview: (":
                if "<UIButtonLabel" in log["receiverSubview"]:
                    regex = "(?<=text = ).*?(?=;)"
                    test_str = log["assert_items"][key]
                    if re.search(regex, test_str):
                        val_to_match = re.search(regex, log["assert_items"][key]).group(0)
                        val_to_match = val_to_match[1:-1]
                        t = Template("XCTAssert(el{{idx}}.exists)").render(idx=i)
                        sentences.append(t)
                        t = Template("""XCTAssert(el{{idx}}.label == "{{val}}")""").render(idx=i, val=val_to_match)
                        sentences.append(t)
            if "Accessibility Label: " in key:
                t = Template("XCTAssert(el{{idx}}.exists)").render(idx=i)
                sentences.append(t)
            if "View Details" == key:
                if "<UILabel:" in log['viewDetails'] or "<UIButtonLabel" in log["receiverSubview"]:
                    # "assert_items": {
                    #                 "View Details": "<UILabel: 0x7fca249084a0;
                    # frame = (37.6667 235; 333.333 58.6667); text = '11';
                    # userInteractionEnabled = NO; layer = <_UILabelLayer: 0x6000037f6cb0>>"
                    regex = "(?<=text = ).*?(?=;)"
                    test_str = log["assert_items"][key]
                    if re.search(regex, test_str):
                        val_to_match = re.search(regex, log["assert_items"][key]).group(0)
                        val_to_match = val_to_match[1:-1]
                        t = Template("""var not_frame{{idx}}: CGRect? = nil""").render(idx=i)
                        sentences.append(t)
                        t = Template("""if app.buttons.staticTexts["{{val_to_match}}"].exists {""").render(idx=i, val_to_match=val_to_match)
                        sentences.append(t)
                        t = Template("""    not_frame{{idx}} = app.buttons.staticTexts["{{val_to_match}}"].frame""").render(idx=i, val_to_match=val_to_match)
                        sentences.append(t)
                        t = Template("""}""").render()
                        sentences.append(t)
                        
                        if (val_to_match[-3:] == "..."):
                            #print(val_to_match)
                            val_to_match = val_to_match[:-3]
                            #print(val_to_match)
                            
                            t = Template("""let el{{idx}}s = app.staticTexts.containing(NSPredicate(format: "label CONTAINS[c] '{{val_to_match}}'"))""").render(idx=i, val_to_match=val_to_match)
                            sentences.append(t)
                            
                        else:
                            
                            t = Template("""let el{{idx}}s = app.staticTexts.containing(NSPredicate(format: "label LIKE[c] '{{val_to_match}}'"))""").render(idx=i, val_to_match=val_to_match)
                            sentences.append(t)


                        t = Template("var correct_el{{idx}}: [XCUIElement] = []").render(idx=i)
                        sentences.append(t)
                        t = Template("for i in 0..<el{{idx}}s.count {").render(idx=i)
                        sentences.append(t)
                        t = Template("    let el = el{{idx}}s.element(boundBy: i)").render(idx=i)
                        sentences.append(t)
                        t = Template("    if (not_frame{{idx}} == nil) || (el.frame != not_frame{{idx}}) {").render(idx=i)
                        sentences.append(t)
                        t = Template("        correct_el{{idx}}.append(el)").render(idx=i)
                        sentences.append(t)
                        t = Template("    }").render()
                        sentences.append(t)
                        t = Template("}").render()
                        sentences.append(t)

                        t = Template("XCTAssert(correct_el{{idx}}.count == 1)").render(idx=i)
                        sentences.append(t)                    
                        t = Template("XCTAssert(correct_el{{idx}}[0].exists)").render(idx=i)
                        sentences.append(t)
                        #t = Template("""XCTAssert(correct_el{{idx}}[0].label == "{{val}}")""").render(idx=i, val=val_to_match)
                        #sentences.append(t)
                
            
    return sentences

In [5]:
def generateTests(logs):
    sentences = []
    for i, log in enumerate(logs):
        print(i)
        el_type = findXCUIElement(log)
        if el_type == "XCUIElementTypeButton":
            t = Template("""let el{{idx}} = app.buttons["{{val}}"]""").render(idx=i, val=getVal(i, log))
            sentences.append(t)
            sentences = evaluateElement(i, log, sentences)
        elif el_type == "XCUIElementTypeStaticText":
            t = Template("""let el{{idx}} = app.staticTexts["{{val}}"]""").render(idx=i, val=getVal(i, log))
            sentences.append(t)
            sentences = evaluateElement(i, log, sentences)
        elif el_type == "XCUIElement":
            t = Template("""let el{{idx}}s = app.cells.containing(NSPredicate(format: "label LIKE[c] '{{val}}'"))""").render(idx=i, val=getVal(i, log))
            sentences.append(t)
            t = Template("""assert(el{{idx}}s.count == 1)""").render(idx=i)
            sentences.append(t)
            t = Template("""let el{{idx}} = el{{idx}}s.element(boundBy: 0)""").render(idx=i)
            sentences.append(t)
            sentences = evaluateElement(i, log, sentences)
        elif el_type == "XCUIElementTypeBackButton":
            t = Template("""let el{{idx}} = app.navigationBars.buttons.element(boundBy: 0)""").render(idx=i)
            sentences.append(t)
            sentences = evaluateElement(i, log, sentences)
    return sentences

In [6]:
sentences = generateTests(logs)
sentences

0
1
2
3
4
5
6
7
8


['let el0 = app.buttons["Add"]',
 'XCTAssertTrue(el0.exists)',
 'el0.tap()',
 'let el1 = app.staticTexts["Add Alarm"]',
 'var not_frame1: CGRect? = nil',
 'if app.buttons.staticTexts["Add Alarm"].exists {',
 '    not_frame1 = app.buttons.staticTexts["Add Alarm"].frame',
 '}',
 'let el1s = app.staticTexts.containing(NSPredicate(format: "label LIKE[c] \'Add Alarm\'"))',
 'var correct_el1: [XCUIElement] = []',
 'for i in 0..<el1s.count {',
 '    let el = el1s.element(boundBy: i)',
 '    if (not_frame1 == nil) || (el.frame != not_frame1) {',
 '        correct_el1.append(el)',
 '    }',
 '}',
 'XCTAssert(correct_el1.count == 1)',
 'XCTAssert(correct_el1[0].exists)',
 'let el2s = app.cells.containing(NSPredicate(format: "label LIKE[c] \'Repeat\'"))',
 'assert(el2s.count == 1)',
 'let el2 = el2s.element(boundBy: 0)',
 'let el3s = app.cells.containing(NSPredicate(format: "label LIKE[c] \'Label\'"))',
 'assert(el3s.count == 1)',
 'let el3 = el3s.element(boundBy: 0)',
 'let el4s = app.cells.cont

In [7]:
def genCode(sentences):
    gen_test = ""
    for s in sentences:
        gen_test = gen_test + "\n            {}".format(s)
    return gen_test
gen_test = genCode(sentences)
print(gen_test)


            let el0 = app.buttons["Add"]
            XCTAssertTrue(el0.exists)
            el0.tap()
            let el1 = app.staticTexts["Add Alarm"]
            var not_frame1: CGRect? = nil
            if app.buttons.staticTexts["Add Alarm"].exists {
                not_frame1 = app.buttons.staticTexts["Add Alarm"].frame
            }
            let el1s = app.staticTexts.containing(NSPredicate(format: "label LIKE[c] 'Add Alarm'"))
            var correct_el1: [XCUIElement] = []
            for i in 0..<el1s.count {
                let el = el1s.element(boundBy: i)
                if (not_frame1 == nil) || (el.frame != not_frame1) {
                    correct_el1.append(el)
                }
            }
            XCTAssert(correct_el1.count == 1)
            XCTAssert(correct_el1[0].exists)
            let el2s = app.cells.containing(NSPredicate(format: "label LIKE[c] 'Repeat'"))
            assert(el2s.count == 1)
            let el2 = el2s.element(boundBy: 0)
            l

In [8]:
def writeToFile(gen_test):
    final = Template("""
    //
    //  AnushkTestSDK-{{sessionNum}}.swift
    //
    //  Automatically created by AnushkTestSDK
    //
    // Created on {{date}} for {{appID}}.
    // 

    import XCTest

    class AnushkTestSDK{{sessionNum}}: XCTestCase {

        func testSession() throws {
            let app = XCUIApplication(bundleIdentifier: "{{appID}}")
            app.launch()        
            {{tests}}

            app.terminate()

        }

    }

    """)

    final = final.render(sessionNum=sessionID,
                         appID=appID,
                         date=date.today(),
                         tests=gen_test
                        )
    print(final)
    f= open("AnushkTestSDK-{}-{}.swift".format(appID, sessionID),"w+")
    f.write(final)
    f.close()

writeToFile(gen_test)


    //
    //  AnushkTestSDK-1591805093.swift
    //
    //  Automatically created by AnushkTestSDK
    //
    // Created on 2020-06-10 for LongGames.Alarm.ios.swift.
    // 

    import XCTest

    class AnushkTestSDK1591805093: XCTestCase {

        func testSession() throws {
            let app = XCUIApplication(bundleIdentifier: "LongGames.Alarm.ios.swift")
            app.launch()        
            
            let el0 = app.buttons["Add"]
            XCTAssertTrue(el0.exists)
            el0.tap()
            let el1 = app.staticTexts["Add Alarm"]
            var not_frame1: CGRect? = nil
            if app.buttons.staticTexts["Add Alarm"].exists {
                not_frame1 = app.buttons.staticTexts["Add Alarm"].frame
            }
            let el1s = app.staticTexts.containing(NSPredicate(format: "label LIKE[c] 'Add Alarm'"))
            var correct_el1: [XCUIElement] = []
            for i in 0..<el1s.count {
                let el = el1s.element(boundBy: i)
           